# Buscar os dados necessários para a análise

## Gerar o dataframe com os dados da Prefeitura

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from io import StringIO
import urllib

### Criar o dataframe que conterá os dados necessários para a análise

In [2]:
df = pd.DataFrame()

### Função para buscar os caminhos para o download de cada dataset

In [3]:
def get_soup(url, search):
    response = requests.get(url)
    
    # Verificar se a conexão foi bem sucedida
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        # Retornar as páginas onde terão as de download
        return soup.find_all('a', href=True, attrs={'title': re.compile(search)}) 
    else:
        return f'Página{link["title"]} está inacessível. Código: {response.status_code}'

### Código para inserir os dados em um dataframe

In [4]:
# URL base do site da Prefeitura
url_base = 'http://dados.prefeitura.sp.gov.br'

# URL inicial onde estarão os links para os datasets
url_inicial = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'

# extensão do dataset para filtrar
file_type = '.csv'

for link in get_soup(url_inicial, r'^Dados do SP156 -.*'):
    
    # Pegar o link de download
    for page in get_soup(url_base+link['href'], r'^http://dados.prefeitura.sp.gov.br/dataset/.*'):
        link_download = page['href']
        
        # Verificar se é o arquivo a ser baixado é Excel
        if file_type in link_download:
            
            # Se for usar .csv
            response_csv = requests.get(page['href'])
            
            if int(link["title"][-4:]) >= 2021:
                df3 = pd.read_csv(StringIO(response_csv.content.decode('cp1252')), sep=';', encoding='utf8', low_memory=False)
                continue
            else:
                df3 = pd.read_csv(StringIO(response_csv.content.decode('utf8')), sep=',', encoding='utf8', low_memory=False)
            
            # Inserir os dados na planilha original
            df = pd.concat([df, df3], ignore_index=True)
            print(f'Os dados do {link["title"]} foram inseridos com sucesso.')

Os dados do Dados do SP156 - Dezembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - Outubro e Novembro - 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 3º TRI 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2020 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2019 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2019 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2018 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2018 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2° SEM 2017 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1° SEM 2017 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1° SEM 2016 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2º SEM 2015 foram inseridos com sucesso.
Os dados do Dados do SP156 - 1º SEM 2015 foram inseridos com sucesso.
Os dados do Dados do SP156 - 2° SEM 2014 foram inseridos com sucesso.
Os

In [5]:
df.sample(20)

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,Canal de atendimento,Especificação do assunto,Bairro
1884154,2020-04-15 13:45:14.383,PORTAL ONLINE,Trânsito e Transporte,Veículos abandonados,Solicitar remoção de veículo/carcaça abandonad...,NaN,NaN,NaN,VILA MARIANA,SAUDE,NaN,NaN,NaN,NaN,2020-07-06 23:47:43.009,FINALIZADA,SMSUB,NaN,NaN,NaN
98638,2020-12-10 03:14:40.213,PORTAL ONLINE,Cidadania e assistência social,Renda Básica Emergencial,Renda Básica Emergencial - Consultar benefício...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-10 03:14:42.569,FINALIZADA,SMADS,NaN,NaN,NaN
7347721,2013-04-05,NaN,NaN,"Jardinagem (poda, corte mato)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-17 00:00:00.000,CANCELADA,SPBT,Central 156,Remocao de arvore,JARDIM BONFIGLIOLI
8071775,2012-03-23 14:14:23.000,NaN,NaN,Buraco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-04-17 13:24:40.010,REALIZADA,SPSA,SAC,Tapa-buraco,SANTO AMARO
6206095,2014-11-06,NaN,NaN,População ou Pessoa em Situação de Rua,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-11-07 00:00:00.000,REALIZADA,SMADS,Central 156,Abordagem social a Pessoa em Situação de Rua,BRAS
5340999,2015-09-03 12:05:58.413,,Rua e bairro,"Calçadas, guias e postes",Calçada particular - denunciar calçada danific...,NaN,NaN,NaN,PIRITUBA-JARAGUA,PIRITUBA,NaN,NaN,NaN,NaN,2015-10-02 19:40:20.887,FINALIZADA,SMSUB,NaN,NaN,NaN
1641794,2020-02-06 19:28:23.066,PRAÇAS DE ATENDIMENTO DAS SUBPREFEITURAS,Trânsito e Transporte,Estacionamento,Cartão de estacionamento idoso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-06 19:28:23.563,FINALIZADA,SMT-DSV,NaN,NaN,NaN
6905853,2013-09-26,NaN,NaN,Obras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-20 00:00:00.000,REALIZADA,SPST,Central 156,Edificacao-vistoria obras/funcionamento,LAUZANE PAULISTA
6905884,2013-09-26,NaN,NaN,Transporte publico / Conduta de Trabalho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-26 00:00:00.000,REALIZADA,SPTRANS,Central 156,Conduta Inadequada do operador,CIDADE SAO MATEUS
7794870,2012-11-25 16:23:23.000,NaN,NaN,Buraco,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,2012-11-30 00:00:00.000,REALIZADA,SPPI,SAC,Tapa-buraco,CERQUEIRA CESAR


## Manter apenas os dados e campos necessários

In [6]:
df = df[['Data de abertura', 'Tema', 'Assunto', 'Especificação do assunto', 'Serviço', 'Bairro', 'Distrito'
         , 'Status da solicitação', 'Data do parecer']]

In [7]:
df = df[df['Assunto'].str.contains('Animais', regex=False) | df['Tema'].str.contains(r'^Anim.*', regex=True)].reset_index(
    drop=True)

In [8]:
df.sample(20)

,Data de abertura,Tema,Assunto,Especificação do assunto,Serviço,Bairro,Distrito,Status da solicitação,Data do parecer
52299,2020-09-17 20:39:00.971,Animais,"Exames, vacinas e castração",NaN,Castrar cães e gatos gratuitamente,NaN,NaN,INDEFERIDO,2020-11-30 13:34:28.525
180895,2017-02-20 19:23:56.245,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,JARDIM HELENA,FINALIZADA,2017-02-24 17:23:00.446
193620,2016-01-11 10:36:56.637,Animais,Animal em via pública,NaN,Avaliação de animal em sofrimento sem propriet...,NaN,CAMPO LIMPO,FINALIZADA,2017-06-19 13:07:38.990
149729,2018-04-07 11:20:45.877,Animais,Criação inadequada de animais,NaN,Denunciar condições inadequadas de criação,NaN,ITAQUERA,FINALIZADA,2018-05-02 11:43:25.307
436902,2012-05-19 11:11:11.023,NaN,Animais / Pombos,Reclamação de Pombos,NaN,CONJUNTO RESIDENCIAL VISTA VERDE,64.0,REALIZADA,2012-06-04 00:00:00.000
118015,2019-06-05 15:55:02.335,Animais,Animal em via pública,NaN,Avaliação de animal em sofrimento sem propriet...,NaN,SAUDE,FINALIZADA,2020-02-14 18:19:17.607
383407,2012-07-25 09:59:07.083,NaN,Animais / Cão,Atropelado ou Acidentado vivo e sem proprietário,NaN,LAUZANE PAULISTA,51.0,CANCELADA,2012-07-25 00:00:00.000
199373,2016-02-13 15:13:44.133,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,ARICANDUVA,FINALIZADA,2016-04-11 13:36:03.013
101548,2019-01-18 18:30:39.845,Animais,Animal em via pública,NaN,Solicitar remoção de animal morto em via pública,NaN,ITAIM BIBI,FINALIZADA,2019-02-06 15:59:39.000
82171,2019-08-09 16:15:04.514,Animais,Animais que transmitem doenças ou risco à saúde,NaN,Abelhas e Vespas - Solicitar vistoria em ninhos,NaN,MOEMA,FINALIZADA,2019-08-13 13:25:07.165


## Gerar o arquivo com os dados necessários

In [10]:
df.to_csv('sp156_all_time.csv', sep=';')